In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pyimzml.ImzMLParser import ImzMLParser
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from datetime import datetime
import gc

In [2]:
# Define folder that contains the preprocessed dhg dataset
DHG_IN_PATH = "/sise/assafzar-group/assafzar/Leor/DHG/Preprocessed"
# Define file to export
META_DATA_PATH = "/sise/assafzar-group/assafzar/Leor/DHG/Preprocessed/Metadata.csv"

In [3]:
# 
meta_data = pd.read_csv(META_DATA_PATH)
# 
meta_data = meta_data[meta_data.file_name.str.contains('r')]
# 
meta_data["label"] = (meta_data.who_grade > 2).astype(int)

In [4]:
#
parsers = { file_name: ImzMLParser(os.path.join(DHG_IN_PATH, f"{file_name}.imzML")) for file_name in meta_data.file_name.unique()}

In [5]:
class DataGenerator(keras.utils.Sequence):
  'Generates data for Keras'
  def __init__(self, df, parsers, batch_size=32,
               dim=(92000,), shuffle=True):
    'Initialization'
    self.dim = dim
    self.batch_size = batch_size
    self.df = df
    self.parsers = parsers
    self.shuffle = shuffle
    self.on_epoch_end()

  def __len__(self):
    'Denotes the number of batches per epoch'
    return max(1, int(np.floor(len(self.df) / self.batch_size)))

  def __getitem__(self, index):
    'Generate one batch of data'
    # Generate indexes of the batch
    batch_df = self.df[index*self.batch_size:(index+1)*self.batch_size]
    # Generate data
    X, y = self.__data_generation(batch_df)
    return X, y

  def on_epoch_end(self):
    'Updates indexes after each epoch'
    if self.shuffle == True:
        self.df = self.df.sample(frac=1).reset_index(drop=True)

  def __data_generation(self, batch_df):
    'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    X = np.empty((self.batch_size, *self.dim))
    y = np.empty((self.batch_size), dtype=int)
    # Generate data
    for i, (_, row) in enumerate(batch_df.iterrows()):
      # get spectra
      _, spectra = self.parsers[row.file_name].getspectrum(row.idx)
      # Store sample
      X[i,] = spectra
      # Store class
      y[i] = row.label
    return X, y

In [6]:
#
now = datetime.now()
#
with open(f"/sise/assafzar-group/assafzar/Leor/log {now.strftime('%d-%m-%Y-%H-%M-%S')}.txt", "w") as file:
  #
  for exclude_sample in tqdm(meta_data.file_name.unique()):
    #
    gc.collect()
    # 
    meta_data_train, meta_data_val = train_test_split(
      meta_data[meta_data.file_name != exclude_sample], test_size=0.2)
    # 
    meta_data_test = meta_data[meta_data.file_name == exclude_sample]

    # Parameters
    params = {"dim": (92000,), "batch_size": 1024,
              "shuffle": True, "parsers": parsers}

    # Generators
    training_generator = DataGenerator(meta_data_train, **params)
    validation_generator = DataGenerator(meta_data_val, **params)
    testing_generator = DataGenerator(meta_data_test, **params)

    #
    model = keras.Sequential(
        [
          keras.Input(shape=(92000,)),
          layers.Dense(512, activation='relu'),
          layers.Dense(512, activation='relu'),
          layers.Dense(256, activation='relu'),
          layers.Dense(256, activation='relu'),
          layers.Dense(128, activation='relu'),
          layers.Dense(1, activation='sigmoid')
        ]
    )
    # 
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # 
    history = model.fit(x=training_generator, validation_data=validation_generator, epochs=10)
    # 
    test_eval = model.evaluate(testing_generator)
    
    #
    file.write(f"Excluded: {exclude_sample}\n")
    file.write(f"Train loss: {history.history['loss']}\n")
    file.write(f"Train accuracy: {history.history['accuracy']}\n")
    file.write(f"Validation loss: {history.history['val_loss']}\n")
    file.write(f"Validation accuracy: {history.history['val_accuracy']}\n")
    file.write(f"Test loss: {test_eval[0]}\n")
    file.write(f"Test accuracy: {test_eval[1]}\n\n")

  0%|          | 0/24 [00:00<?, ?it/s]2022-04-24 21:23:00.359591: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-24 21:23:00.359626: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-24 21:23:00.359652: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cs-cpu-01): /proc/driver/nvidia/version does not exist
2022-04-24 21:23:00.398812: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-24 21:23:06.601960: I tensorflow/compiler/mlir/ml

Epoch 1/10
32/32 [==============================] - 259s 8s/step - loss: 0.6867 - accuracy: 0.5466 - val_loss: 0.6697 - val_accuracy: 0.5598
Epoch 2/10
32/32 [==============================] - 251s 8s/step - loss: 0.4995 - accuracy: 0.7667 - val_loss: 0.3098 - val_accuracy: 0.9329
Epoch 3/10
32/32 [==============================] - 249s 8s/step - loss: 0.2235 - accuracy: 0.9353 - val_loss: 0.1831 - val_accuracy: 0.9426
Epoch 4/10
32/32 [==============================] - 244s 8s/step - loss: 0.1748 - accuracy: 0.9446 - val_loss: 0.1780 - val_accuracy: 0.9437
Epoch 5/10
32/32 [==============================] - 244s 8s/step - loss: 0.1657 - accuracy: 0.9456 - val_loss: 0.1584 - val_accuracy: 0.9437
Epoch 6/10
32/32 [==============================] - 242s 8s/step - loss: 0.1015 - accuracy: 0.9458 - val_loss: 0.0585 - val_accuracy: 0.9438
Epoch 7/10
32/32 [==============================] - 245s 8s/step - loss: 0.0350 - accuracy: 0.9877 - val_loss: 0.0203 - val_accuracy: 0.9961
Epoch 8/10
32

  4%|▍         | 1/24 [41:43<15:59:40, 2503.51s/it]

Epoch 1/10
33/33 [==============================] - 270s 8s/step - loss: 0.6792 - accuracy: 0.5642 - val_loss: 0.6180 - val_accuracy: 0.5782
Epoch 2/10
33/33 [==============================] - 265s 8s/step - loss: 0.3962 - accuracy: 0.8513 - val_loss: 0.2178 - val_accuracy: 0.9401
Epoch 3/10
33/33 [==============================] - 267s 8s/step - loss: 0.2101 - accuracy: 0.9322 - val_loss: 0.1877 - val_accuracy: 0.9381
Epoch 4/10
33/33 [==============================] - 254s 8s/step - loss: 0.1715 - accuracy: 0.9442 - val_loss: 0.1718 - val_accuracy: 0.9424
Epoch 5/10
33/33 [==============================] - 258s 8s/step - loss: 0.1505 - accuracy: 0.9474 - val_loss: 0.1387 - val_accuracy: 0.9449
Epoch 6/10
33/33 [==============================] - 259s 8s/step - loss: 0.1050 - accuracy: 0.9475 - val_loss: 0.0789 - val_accuracy: 0.9442
Epoch 7/10
33/33 [==============================] - 257s 8s/step - loss: 0.0526 - accuracy: 0.9681 - val_loss: 0.0508 - val_accuracy: 0.9899
Epoch 8/10
33

  8%|▊         | 2/24 [1:25:13<15:29:39, 2535.44s/it]

...

 92%|█████████▏| 22/24 [15:47:25<1:26:16, 2588.47s/it]

Epoch 1/10
31/31 [==============================] - 249s 8s/step - loss: 0.6885 - accuracy: 0.5425 - val_loss: 0.6768 - val_accuracy: 0.5516
Epoch 2/10
31/31 [==============================] - 246s 8s/step - loss: 0.5078 - accuracy: 0.7498 - val_loss: 0.3319 - val_accuracy: 0.9314
Epoch 3/10
31/31 [==============================] - 236s 8s/step - loss: 0.2265 - accuracy: 0.9412 - val_loss: 0.1980 - val_accuracy: 0.9371
Epoch 4/10
31/31 [==============================] - 278s 9s/step - loss: 0.1747 - accuracy: 0.9451 - val_loss: 0.1909 - val_accuracy: 0.9361
Epoch 5/10
31/31 [==============================] - 236s 8s/step - loss: 0.1626 - accuracy: 0.9451 - val_loss: 0.1513 - val_accuracy: 0.9382
Epoch 6/10
31/31 [==============================] - 234s 8s/step - loss: 0.0945 - accuracy: 0.9452 - val_loss: 0.0695 - val_accuracy: 0.9408
Epoch 7/10
31/31 [==============================] - 247s 8s/step - loss: 0.0408 - accuracy: 0.9705 - val_loss: 0.0172 - val_accuracy: 0.9957
Epoch 8/10
31

 96%|█████████▌| 23/24 [16:28:32<42:32, 2552.17s/it]  

Epoch 1/10
32/32 [==============================] - 236s 7s/step - loss: 0.6859 - accuracy: 0.5556 - val_loss: 0.6761 - val_accuracy: 0.5631
Epoch 2/10
32/32 [==============================] - 239s 7s/step - loss: 0.5344 - accuracy: 0.7289 - val_loss: 0.3591 - val_accuracy: 0.9285
Epoch 3/10
32/32 [==============================] - 244s 8s/step - loss: 0.2466 - accuracy: 0.9352 - val_loss: 0.1854 - val_accuracy: 0.9435
Epoch 4/10
32/32 [==============================] - 246s 8s/step - loss: 0.1733 - accuracy: 0.9457 - val_loss: 0.1761 - val_accuracy: 0.9443
Epoch 5/10
32/32 [==============================] - 251s 8s/step - loss: 0.1661 - accuracy: 0.9465 - val_loss: 0.1675 - val_accuracy: 0.9437
Epoch 6/10
32/32 [==============================] - 255s 8s/step - loss: 0.1261 - accuracy: 0.9466 - val_loss: 0.0755 - val_accuracy: 0.9443
Epoch 7/10
32/32 [==============================] - 247s 8s/step - loss: 0.0415 - accuracy: 0.9778 - val_loss: 0.0248 - val_accuracy: 0.9940
Epoch 8/10
32

100%|██████████| 24/24 [17:09:52<00:00, 2530.43s/it]


In [14]:
output = []

with open(f"/sise/assafzar-group/assafzar/Leor/log {now.strftime('%d-%m-%Y-%H-%M-%S')}.txt", "r") as file:
  for _ in tqdm(meta_data.file_name.unique()):
    exclude_sample = file.readline().replace("Excluded: ", "").replace("\n","")
    train_loss = list(file.readline().replace("Train loss: ", "").replace("[", "").replace("]", "").replace("\n","").split(","))[-1]
    train_accuracy = list(file.readline().replace("Train accuracy: ", "").replace("[", "").replace("]", "").replace("\n","").split(","))[-1]
    val_loss = list(file.readline().replace("Validation  loss: ", "").replace("[", "").replace("]", "").replace("\n","").split(","))[-1]
    val_accuracy = list(file.readline().replace("Validation accuracy: ", "").replace("[", "").replace("]", "").replace("\n","").split(","))[-1]
    test_loss = file.readline().replace("Test loss: ", "").replace("\n","")
    test_accuracy = file.readline().replace("Test accuracy: ", "").replace("\n","")
    _ = file.readline()
    output.append([exclude_sample, float(train_loss), float(train_accuracy), float(val_loss), float(val_accuracy), float(test_loss), float(test_accuracy)])
    
new_df = pd.DataFrame(columns=["exclude_sample", "train_loss", "train_accuracy", "val_loss", "val_accuracy", "test_loss", "test_accuracy"], data=output)

100%|██████████| 24/24 [00:00<00:00, 26262.27it/s]


In [15]:
new_df[["train_accuracy", "val_accuracy", "test_accuracy"]].mean() * 100

train_accuracy    98.109323
val_accuracy      98.044840
test_accuracy     74.058024
dtype: float64

In [17]:
new_df[["train_accuracy", "val_accuracy", "test_accuracy"]].sem() * 100

train_accuracy    1.791221
val_accuracy      1.795072
test_accuracy     7.973695
dtype: float64

In [18]:
from sklearn.dummy import DummyClassifier

output2 = []

for exclude_sample in tqdm(meta_data.file_name.unique()):
    #
    gc.collect()
    # 
    meta_data_train, meta_data_val = train_test_split(
      meta_data[meta_data.file_name != exclude_sample], test_size=0.2)
    # 
    meta_data_test = meta_data[meta_data.file_name == exclude_sample]
    
    #
    X_train = meta_data_train.drop(["label", "histology", "who_grade"], axis=1)
    Y_train = meta_data_train["label"]
    
    #
    X_test = meta_data_test.drop(["label", "histology", "who_grade"], axis=1)
    Y_test = meta_data_test["label"]
    
    # 
    dummy_clf = DummyClassifier(strategy="most_frequent")
    
    # 
    dummy_clf.fit(X_train, Y_train)
    
    #
    output2.append([dummy_clf.score(X_train, Y_train), dummy_clf.score(X_test, Y_test)])

100%|██████████| 24/24 [00:06<00:00,  3.51it/s]


In [19]:
new_df_2 = pd.DataFrame(columns=["dummy_train_accuracy", "dummy_test_accuracy"], data=output2)

In [20]:
new_df_2[["dummy_train_accuracy", "dummy_test_accuracy"]].mean() * 100

dummy_train_accuracy    57.55302
dummy_test_accuracy     62.50000
dtype: float64

In [21]:
new_df_2[["dummy_train_accuracy", "dummy_test_accuracy"]].sem() * 100

dummy_train_accuracy     0.488037
dummy_test_accuracy     10.094661
dtype: float64